<center><img src="https://drive.google.com/uc?export=view&id=1ygAs8EMNlIim2ypwmvQn9yN1LbY3hWHV" alt="Drawing"  width="30%"/><center>

# <center><strong>Development notebook</strong></center>
<br/>

<br/><center>This notebook allows you to visualize the data used in the **FLAIR #2 challenge**.<br/>The code bellow works with the toy dataset (subset) provided in the starting-kit alongside this notebook as well as with the full FLAIR-two dataset accessible after registration to the competition.</center> <br/> 
<center>**We also strongly advise you to read the data technical description provided in the datapaper.**</center>
<br/> <br/> 
  


Handle all the generic imports

In [ ]:
import yaml
import sys
import torch

import numpy as np
import matplotlib.pyplot as plt
import torchvision.transforms as transforms

from sklearn.cluster import HDBSCAN
from os.path import join
from pathlib import Path
from importlib import reload

Import code from or based upon the FLAIR-2 reference implementation 

In [ ]:
FLAIR_path = join(Path.cwd().parents[0],'FLAIR-code/src')
if FLAIR_path not in sys.path:
    sys.path.append(FLAIR_path)

from data_display import (display_nomenclature,
                            display_samples, 
                            display_time_serie,
                            display_all_with_semantic_class, 
                            display_all, 
                            read_dates, 
                            filter_dates)
from load_data import load_data
from FusedDataset import FusedDataset
from calc_miou import calc_miou


## <font color='#90c149'>Load Data</font>

<br/><hr>

Use reference code to create lists containing the paths to the input images (`images`) and supervision masks (`masks`) files of the dataset.<hr><br/>

In [ ]:
config_path = "/app/FLAIR-HDBSCAN/flair-2-config.yml" 
with open(config_path, "r") as f:
    config = yaml.safe_load(f)

# Creation of the train, val, and test dictionaries with the data file paths
# Note that due to using the toy dataset we assign 100% of the data for training. 
# While not best practice for machine learning, for the toy dataset when using the stock code and less than 100%, issues randomly arise when using the reference loader
# as the random selection can result in some semantic classes not being represented. If the full FLAIR #2 dataset were employed, validation data should be separate. 
# Due to size limitations on HDBSCAN, for actual training we downsample to ~1 % of the training data for training and use 100% of the training data for fitting. 
d_train, d_val, d_test = load_data(config, val_percent=1)

# Convert to torch Datasets
train_dataset = FusedDataset(dict_files=d_train, config=config)
valid_dataset = FusedDataset(dict_files=d_val, config=config)
test_dataset = FusedDataset(dict_files=d_val, config=config)

<br><br>
<hr style="height:3px;border-width:0;color:red;background-color:red">   

# <center><font color='red'>PART-2: Development area</font></center>

<br/><hr>

Area under active development. 

<hr><br/>

In [ ]:
project_path = join(Path.cwd().parents[0],'code')
if project_path not in sys.path:
    sys.path.append(project_path)

In [ ]:
import display
reload(display)
from display import display_confusion

import classifier
reload(classifier)
from classifier import train_and_validate_model

### <br><br>
<hr style="height:3px;border-width:0;color:red;background-color:red">   

# <center><font color='red'>PART-12: Data Fusion - Spectral Normalization</font></center>

<br/><hr>
This section shows the effects of data fusion of satellite imagery with aerial imagery combined with spectral normalization
<br/> 

<hr><br/>

In [ ]:
%%time
knn_fusion_normalized = train_and_validate_model(train_dataset, config, use_satellite=True, use_hdbscan=False, robust_scale=False, scale_by_intensity=True, append_intensity=True)

In [ ]:
display_confusion(knn_fusion_normalized['true_classes'], knn_fusion_normalized['predicted_classes'], config, 'KNN Fusion w/ Spectral Normalization')

In [ ]:
%reset_selective -f knn_fusion_normalized

In [ ]:
%%time
hdbscan_fusion_normalized = train_and_validate_model(train_dataset, config, use_satellite=True, use_hdbscan=True, robust_scale=False, scale_by_intensity=True, append_intensity=True)

In [ ]:
display_confusion(hdbscan_fusion_normalized['true_classes'], hdbscan_fusion_normalized['predicted_classes'], config, 'HDBSCAN Fusion w/ Spectral Normalization')

In [ ]:
%reset_selective -f hdbscan_fusion_normalized